In [7]:
from kaggle_environments import make
from keras import Model
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.processors import Processor
from keras.layers import Dense, Dropout, Input, Reshape
import numpy as np

In [31]:
class ConnectXProcessor(Processor):
    def process_observation(self, observation):
        return np.array(observation['board'])

    def process_state_batch(self, batch):
        return batch

    def process_reward(self, _reward):
        return (10 if _reward == 1 else 0 if _reward == 0 else -10) if _reward is not None else -10

    def process_action(self, action):
        return int(action)

In [32]:
i = Input(shape=(1, 42))
x = Reshape((42,))(i)
x = Dense(42, activation='leaky_relu')(x)
x = Dropout(0.1)(x)
x = Dense(42, activation='leaky_relu')(x)
x = Dropout(0.1)(x)
x = Dense(24, activation='linear')(x)
x = Dense(24, activation='linear')(x)
o = Dense(7, activation='leaky_relu')(x)
model = Model(inputs=i, outputs=o)
print(model.layers[-2])
print(model.output.shape)

(None, 7)


In [33]:
policy = EpsGreedyQPolicy(0.2)
processor = ConnectXProcessor()
memory = SequentialMemory(limit=50000, window_length=1)
    agent = DQNAgent(model=model, policy=policy, memory=memory, nb_actions=7, nb_steps_warmup=100, target_model_update=1e-2,
                 processor=processor, enable_double_dqn=True, enable_dueling_network=True)
agent.compile(optimizer=Adam(), metrics=['mae'])
# agent.load_weights('dqn_weights_23.h5f')

In [20]:
env = make("connectx", debug=False)
trainer = env.train([None, "random"])
agent.fit(trainer, nb_steps=50000, visualize=False, verbose=1, nb_max_episode_steps=1000)
agent.save_weights('dqn_weights.h5f', overwrite=True)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 128s 13ms/step - reward: 0.5338
1160 episodes - episode_reward: 4.599 [-8.000, 12.000] - loss: 0.920 - mae: 2.547 - mean_q: 3.379

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 126s 13ms/step - reward: 0.5716
1133 episodes - episode_reward: 5.046 [-6.500, 12.000] - loss: 0.779 - mae: 3.146 - mean_q: 3.909

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 128s 13ms/step - reward: 0.5439
1130 episodes - episode_reward: 4.815 [-7.500, 12.500] - loss: 0.754 - mae: 3.319 - mean_q: 4.060

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 133s 13ms/step - reward: 0.5457
1131 episodes - episode_reward: 4.824 [-6.500, 11.500] - loss: 0.748 - mae: 3.309 - mean_q: 4.005

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 135s 14ms/step - reward: 0.5249
done, too

In [22]:
agent.fit(trainer, nb_steps=50000, visualize=False, verbose=1, nb_max_episode_steps=1000)
agent.save_weights('dqn_weights_2.h5f', overwrite=True)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 131s 13ms/step - reward: 0.5540
1172 episodes - episode_reward: 4.725 [-6.500, 12.000] - loss: 0.776 - mae: 3.367 - mean_q: 4.028

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 133s 13ms/step - reward: 0.5342
1135 episodes - episode_reward: 4.705 [-7.500, 11.500] - loss: 0.793 - mae: 3.362 - mean_q: 4.013

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 132s 13ms/step - reward: 0.5030
1068 episodes - episode_reward: 4.709 [-8.000, 12.000] - loss: 0.815 - mae: 3.389 - mean_q: 4.047

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 132s 13ms/step - reward: 0.5139
1095 episodes - episode_reward: 4.695 [-7.000, 12.000] - loss: 0.831 - mae: 3.384 - mean_q: 4.046

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 131s 13ms/step - reward: 0.5131
done, too

In [28]:
agent.load_weights('dqn_weights_2.h5f')
agent.fit(trainer, nb_steps=50000, visualize=False, verbose=1, nb_max_episode_steps=1000)
agent.save_weights('dqn_weights_3.h5f', overwrite=True)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 135s 14ms/step - reward: 0.4859
1131 episodes - episode_reward: 4.293 [-7.500, 11.500] - loss: 0.841 - mae: 3.287 - mean_q: 4.080

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 135s 13ms/step - reward: 0.4907
1200 episodes - episode_reward: 4.090 [-7.000, 12.500] - loss: 0.895 - mae: 3.204 - mean_q: 3.930

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 131s 13ms/step - reward: 0.4833
1188 episodes - episode_reward: 4.070 [-7.500, 11.500] - loss: 0.897 - mae: 3.091 - mean_q: 3.772

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 141s 14ms/step - reward: 0.4868
1188 episodes - episode_reward: 4.095 [-7.500, 12.000] - loss: 0.863 - mae: 3.031 - mean_q: 3.690

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 134s 13ms/step - reward: 0.4832
done, too

In [ ]:
for c in range(3,23):
    env = make("connectx", debug=False)
    trainer = env.train([None, "negamax"])
    if c>0:
        agent.load_weights(f'dqn_weights_{c}.h5f')
    agent.fit(trainer, nb_steps=10000, visualize=False, verbose=1, nb_max_episode_steps=1000)
    agent.save_weights(f'dqn_weights_{c+1}.h5f', overwrite=True)

Training for 10000 steps ...
Interval 1 (0 steps performed)


C:\Users\Martin\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


10000/10000 [==============================] - 632s 63ms/step - reward: 0.1283
done, took 632.274 seconds
Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 596s 60ms/step - reward: 0.1421
done, took 596.335 seconds
Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 578s 58ms/step - reward: 0.1563
done, took 578.387 seconds
Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 577s 58ms/step - reward: 0.1431
done, took 577.228 seconds
Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 579s 58ms/step - reward: 0.1421
done, took 579.499 seconds
Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 598s 60ms/step - reward: 0.1439
done, took 598.367 seconds
Training for 10000 steps ...
Interval 1 (0 steps performed)
1000

In [6]:
def kaggle_agent(observation, _):
    return processor.process_action(np.argmax(agent.forward(processor.process_observation(observation))))

In [ ]:
env = make("connectx", debug=True)
env.play([kaggle_agent, None], width=500, height=450, fps=10)
# env.render(mode="ipython", width=500, height=450)

In [36]:
save_name = "dqn_weights_dd"
for c in range(1,20):
    env = make("connectx", debug=False)
    trainer = env.train([None, "negamax"])
    if c>0:
        agent.load_weights(f'{save_name}_{c}.h5f')
    agent.fit(trainer, nb_steps=5000, visualize=False, verbose=1, nb_max_episode_steps=1000)
    agent.save_weights(f'{save_name}_{c+1}.h5f', overwrite=True)

Training for 5000 steps ...
Interval 1 (0 steps performed)
 5000/10000 [==============>...............] - ETA: 5:50 - reward: -0.7560done, took 350.140 seconds
Training for 5000 steps ...
Interval 1 (0 steps performed)
 4998/10000 [=============>................] - ETA: 5:41 - reward: -0.6643done, took 341.624 seconds
Training for 5000 steps ...
Interval 1 (0 steps performed)
 5000/10000 [==============>...............] - ETA: 5:35 - reward: -0.6840done, took 335.882 seconds
Training for 5000 steps ...
Interval 1 (0 steps performed)
 5000/10000 [==============>...............] - ETA: 5:36 - reward: -0.6320done, took 336.635 seconds
Training for 5000 steps ...
Interval 1 (0 steps performed)
 5000/10000 [==============>...............] - ETA: 5:31 - reward: -0.6520done, took 331.924 seconds
Training for 5000 steps ...
Interval 1 (0 steps performed)
  135/10000 [..............................] - ETA: 9:59 - reward: -0.7407 done, took 8.299 seconds
Training for 5000 steps ...
Interval 1 (0

In [37]:
save_name = "dqn_weights_dd"
agent.policy = EpsGreedyQPolicy(0.05)
for c in range(6,20):
    env = make("connectx", debug=False)
    trainer = env.train([None, "negamax"])
    if c>0:
        agent.load_weights(f'{save_name}_{c}.h5f')
    agent.fit(trainer, nb_steps=2500, visualize=False, verbose=1, nb_max_episode_steps=1000)
    agent.save_weights(f'{save_name}_{c+1}.h5f', overwrite=True)

Training for 2500 steps ...
Interval 1 (0 steps performed)
 2500/10000 [======>.......................] - ETA: 7:50 - reward: -0.4440done, took 156.897 seconds
Training for 2500 steps ...
Interval 1 (0 steps performed)
 2498/10000 [======>.......................] - ETA: 7:34 - reward: -0.2842done, took 151.545 seconds
Training for 2500 steps ...
Interval 1 (0 steps performed)
 2500/10000 [======>.......................] - ETA: 7:46 - reward: -0.3400done, took 155.730 seconds
Training for 2500 steps ...
Interval 1 (0 steps performed)
 2500/10000 [======>.......................] - ETA: 7:38 - reward: -0.3760done, took 153.065 seconds
Training for 2500 steps ...
Interval 1 (0 steps performed)
 2500/10000 [======>.......................] - ETA: 7:28 - reward: -0.2680done, took 149.445 seconds
Training for 2500 steps ...
Interval 1 (0 steps performed)
 2498/10000 [======>.......................] - ETA: 8:04 - reward: -0.2042done, took 161.427 seconds
Training for 2500 steps ...
Interval 1 (

In [38]:
save_name = "dqn_weights_dd"
agent.policy = EpsGreedyQPolicy(0.025)
for c in range(15,20):
    env = make("connectx", debug=False)
    trainer = env.train([None, "negamax"])
    if c>0:
        agent.load_weights(f'{save_name}_{c}.h5f')
    agent.fit(trainer, nb_steps=2500, visualize=False, verbose=1, nb_max_episode_steps=1000)
    agent.save_weights(f'{save_name}_{c+1}.h5f', overwrite=True)

Training for 2500 steps ...
Interval 1 (0 steps performed)
 2498/10000 [======>.......................] - ETA: 7:07 - reward: -0.0721done, took 142.469 seconds
Training for 2500 steps ...
Interval 1 (0 steps performed)
 2500/10000 [======>.......................] - ETA: 7:07 - reward: -0.0720done, took 142.519 seconds
Training for 2500 steps ...
Interval 1 (0 steps performed)
 2500/10000 [======>.......................] - ETA: 7:08 - reward: -0.1280done, took 142.843 seconds
Training for 2500 steps ...
Interval 1 (0 steps performed)
 2497/10000 [======>.......................] - ETA: 7:13 - reward: -0.0561done, took 144.540 seconds
Training for 2500 steps ...
Interval 1 (0 steps performed)
 2499/10000 [======>.......................] - ETA: 7:04 - reward: 0.0360done, took 141.459 seconds


In [39]:
save_name = "dqn_weights_dd"
agent.policy = EpsGreedyQPolicy(0.0125)
for c in range(20,25):
    env = make("connectx", debug=False)
    trainer = env.train([None, "negamax"])
    if c>0:
        agent.load_weights(f'{save_name}_{c}.h5f')
    agent.fit(trainer, nb_steps=2000, visualize=False, verbose=1, nb_max_episode_steps=1000)
    agent.save_weights(f'{save_name}_{c+1}.h5f', overwrite=True)

Training for 2000 steps ...
Interval 1 (0 steps performed)
 1999/10000 [====>.........................] - ETA: 7:40 - reward: 0.0850done, took 115.193 seconds
Training for 2000 steps ...
Interval 1 (0 steps performed)
 2000/10000 [=====>........................] - ETA: 7:37 - reward: -0.0300done, took 114.517 seconds
Training for 2000 steps ...
Interval 1 (0 steps performed)
 2000/10000 [=====>........................] - ETA: 7:43 - reward: -0.0350done, took 115.953 seconds
Training for 2000 steps ...
Interval 1 (0 steps performed)
 1999/10000 [====>.........................] - ETA: 7:22 - reward: -0.0500done, took 110.760 seconds
Training for 2000 steps ...
Interval 1 (0 steps performed)
 2000/10000 [=====>........................] - ETA: 7:29 - reward: -0.0050done, took 112.417 seconds


In [40]:
save_name = "dqn_weights_dd"
agent.policy = EpsGreedyQPolicy(0.1)
for c in range(25,30):
    env = make("connectx", debug=False)
    trainer = env.train([None, "negamax"])
    agent.load_weights(f'{save_name}_{c}.h5f')
    agent.fit(trainer, nb_steps=2500, visualize=False, verbose=1, nb_max_episode_steps=1000)
    agent.save_weights(f'{save_name}_{c+1}.h5f', overwrite=True)

Training for 2500 steps ...
Interval 1 (0 steps performed)
 2498/10000 [======>.......................] - ETA: 9:00 - reward: -0.3403done, took 179.968 seconds
Training for 2500 steps ...
Interval 1 (0 steps performed)
 2500/10000 [======>.......................] - ETA: 9:05 - reward: -0.2400done, took 182.092 seconds
Training for 2500 steps ...
Interval 1 (0 steps performed)
 2499/10000 [======>.......................] - ETA: 8:48 - reward: -0.2601done, took 176.276 seconds
Training for 2500 steps ...
Interval 1 (0 steps performed)
 2500/10000 [======>.......................] - ETA: 7:45 - reward: -0.2840done, took 155.157 seconds
Training for 2500 steps ...
Interval 1 (0 steps performed)
 2498/10000 [======>.......................] - ETA: 7:39 - reward: -0.3082done, took 153.125 seconds


In [1]:
from kaggle_environments import make

Loading environment lux_ai_s2 failed: No module named 'pettingzoo'


In [2]:
env = make("connectx", debug=True)

In [3]:
trainer = env.train([None, "negamax"])

In [4]:
trainer.reset()

{'remainingOverageTime': 60,
 'step': 0,
 'board': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'mark': 1}

In [5]:
env.reset()

[{'action': 0,
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60,
   'step': 0,
   'board': [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   'mark': 1},
  'status': 'ACTIVE'},
 {'action': 0,
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'mark': 2},
  'status': 'INACTIVE'}]

In [81]:
ep = env.run([ lambda _,__:0,"random"])
[(x[0]['reward'], x[1]['reward']) for x in ep][-1]

Invalid Action: Invalid column: 0


(None, 0)

[(0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, -1)]